In [1]:
import numpy as np
import pandas as pd
from keras.layers import Input, Dense, Flatten,Conv1D, Bidirectional, LSTM, MaxPooling1D, BatchNormalization, Dropout, Layer
from keras.models import Sequential
from keras.losses import BinaryCrossentropy
from keras.optimizers import Adam
from keras.metrics import Precision, Recall
from keras import backend as K
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
X_data = np.load("Processed DaphNet/final_x.npy")
Y_data = np.load("Processed DaphNet/final_y.npy")

X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=42)



print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


(28424, 256, 3) (28424, 1)
(7107, 256, 3) (7107, 1)


In [3]:
class Attention(Layer):
    
    def __init__(self, return_sequences=True):
        self.return_sequences = return_sequences
        super(Attention,self).__init__()
        
    def build(self, input_shape):
        
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")
        
        super(Attention,self).build(input_shape)
        
    def call(self, x):
        
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a
        
        if self.return_sequences:
            return output
        
        return K.sum(output, axis=1)

def Specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

In [5]:
BATCH_SIZE = 32

model= Sequential()

model.add(Input(shape=[256, 3]))

model.add(Conv1D(filters=64, kernel_size=3))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=64, kernel_size=3))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=128, kernel_size=3))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))

model.add(Bidirectional(LSTM(units=128, return_sequences=True)))
model.add(Dropout(0.2))

model.add(Attention(return_sequences=False))

model.add(Flatten())
model.add(Dense(units=64, activation="relu"))

model.add(Dense(units=1, activation="sigmoid"))

model.compile(optimizer=Adam(learning_rate=0.01), loss=BinaryCrossentropy(), metrics=["binary_accuracy", "AUC", Recall(), Specificity, Precision()])



In [6]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 254, 64)           640       
                                                                 
 batch_normalization_3 (Batc  (None, 254, 64)          256       
 hNormalization)                                                 
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 127, 64)          0         
 1D)                                                             
                                                                 
 conv1d_4 (Conv1D)           (None, 125, 64)           12352     
                                                                 
 batch_normalization_4 (Batc  (None, 125, 64)          256       
 hNormalization)                                                 
                                                      

In [7]:
model.fit(x=X_train, y=y_train, batch_size=BATCH_SIZE, epochs=14, validation_data=(X_test, y_test))

Epoch 1/14
889/889 [==============================] - 141s 143ms/step - loss: 0.2676 - binary_accuracy: 0.8862 - auc: 0.8902 - recall: 0.3935 - Specificity: 0.9698 - precision: 0.6879 - val_loss: 0.2460 - val_binary_accuracy: 0.8888 - val_auc: 0.9200 - val_recall: 0.3392 - val_Specificity: 0.9866 - val_precision: 0.8180
Epoch 2/14
889/889 [==============================] - 121s 136ms/step - loss: 0.2194 - binary_accuracy: 0.9058 - auc: 0.9329 - recall: 0.5856 - Specificity: 0.9604 - precision: 0.7134 - val_loss: 0.2656 - val_binary_accuracy: 0.8756 - val_auc: 0.9407 - val_recall: 0.8779 - val_Specificity: 0.8760 - val_precision: 0.5558
Epoch 3/14
889/889 [==============================] - 125s 140ms/step - loss: 0.2075 - binary_accuracy: 0.9102 - auc: 0.9401 - recall: 0.6012 - Specificity: 0.9627 - precision: 0.7316 - val_loss: 0.3101 - val_binary_accuracy: 0.8423 - val_auc: 0.9443 - val_recall: 0.9115 - val_Specificity: 0.8289 - val_precision: 0.4880
Epoch 4/14
889/889 [==============

In [8]:
model.predict(X_test[0].reshape(1, 256, 3))

1/1 [==============================] - 2s 2s/step


array([[7.137048e-08]], dtype=float32)

In [9]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False

tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\patel_223gv41\AppData\Local\Temp\tmp4zoun_gg\assets


INFO:tensorflow:Assets written to: C:\Users\patel_223gv41\AppData\Local\Temp\tmp4zoun_gg\assets


In [10]:
with open('models/baseline_model.tflite', 'wb') as f:
  f.write(tflite_model)

In [15]:
from sklearn.metrics import precision_recall_curve, roc_curve

In [16]:
y_pred = model.predict(X_test)

# precision, recall, thresholds = precision_recall_curve(y_test, y_pred)
fpr, tpr, thresholds = roc_curve(y_test, y_pred)

223/223 [==============================] - 9s 42ms/step


In [17]:
# Calculate the G-mean
gmean = np.sqrt(tpr * (1 - fpr))

# Find the optimal threshold
index = np.argmax(gmean)
thresholdOpt = round(thresholds[index], ndigits = 4)
gmeanOpt = round(gmean[index], ndigits = 4)
fprOpt = round(fpr[index], ndigits = 4)
tprOpt = round(tpr[index], ndigits = 4)
print('Best Threshold: {} with G-Mean: {}'.format(thresholdOpt, gmeanOpt))
print('FPR: {}, TPR: {}'.format(fprOpt, tprOpt))
print('Sensitivity: {}, Specificity: {}'.format(tprOpt, 1 - fprOpt))


Best Threshold: 0.25189998745918274 with G-Mean: 0.894
FPR: 0.1001, TPR: 0.8882
